In [1]:
from perceiver.data import *
import utils
import numpy as np
from sklearn import metrics

/home/nick/anaconda3/envs/perceiver-io/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dm = CosmicDataModule(batch_size=100, root="../datasets")
dm.setup(stage=None)
x_test, y_test = utils.dm_to_numpy(dm.test_dataloader())

../datasets/cosmic
../datasets/cosmic/npy_train/f435w/9490/a3/2_208.npy
49
(4830, 16384) (4830, 16384)


In [3]:
dl = dm.test_dataloader()


In [4]:
imgs, masks, ignores = [], [], []
for batch in dl:
    img, mask, ignore = batch
    imgs.append(img.numpy())
    masks.append(mask.numpy())
    ignores.append(ignore.numpy())


In [5]:
imgs = np.concatenate(imgs)
masks = np.concatenate(masks)
ignores = np.concatenate(ignores)

In [6]:
print(imgs.shape)

(4830, 128, 128)


# Load the model

In [7]:
import xgboost as xgb

def eval_cosmic_seed(seed):
    model = xgb.XGBClassifier()
    model.load_model(f"../xgboost_model_cosmic_{seed}.json")
    logits = model.predict([x.reshape(-1) for x in imgs]).reshape(*imgs.shape)
    logits = logits.flatten()

    test_predictions = (logits.reshape(-1, 1, 128, 128) > 0.5).flatten()
    test_gts = masks.reshape(-1, 1, 128, 128).flatten()

    print(test_predictions.shape)
    print(test_gts.shape)


    test_predictions = test_predictions.astype(np.int32)
    test_gts = test_gts.astype(np.int32)
    auroc = metrics.roc_auc_score(test_gts, test_predictions)

    return 1 - auroc

In [8]:
res = [eval_cosmic_seed(i) for i in range(3)]
res

(79134720,)
(79134720,)
(79134720,)
(79134720,)
(79134720,)
(79134720,)


[0.2322921349727951, 0.23098545855749997, 0.23206601628611412]

In [9]:
res = np.array(res) * 100
mean = np.mean(res)
std = np.std(res)

print(mean, std)

23.178120327213637 0.057019859304774476
